# Supplementary: Counts of differentially expressed genes per examined tissue

This notebook aggregates the results from the differential gene expression (**see** [figure1.ipynb](figure1.ipynb)), and more specifically the `limma::topTable()` output dataframes across all tissues in the GTEX cohort and generates summary statistics for the number of genes found to be statistically up or downregulated between male and female subjects.

 ---
 
 **Running this notebook**:
 
A few steps are needed before you can run this document on your own. The GitHub repository (https://github.com/TheJacksonLaboratory/sbas) of the project contains detailed instructions for setting up the environment in the **`dependencies/README.md`** document. Before starting with the analysis, make sure you have first completed the dependencies set up by following the instructions described there. If you have not done this already, you will need to close and restart this notebook before running it.

All paths defined in this Notebook are relative to the parent directory (repository). 

 ---


# Loading dependencies

In [1]:
library(dplyr)
library(tidyr)
library(reshape)
library(ggplot2)
# Install this version: > devtools::install_github("ropensci/piggyback@87f71e8", upgrade="never")
library(piggyback)
library(snakecase)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




ERROR: Error in library(reshape): there is no package called ‘reshape’


# Retrieving the results from the Differential Gene Expression using [`ropensci/piggyback`](https://github.com/ropensci/piggyback)

This notebook requires as input data the limma `topTable()` objects from the Differential Gene Expression analysis (see [figure1.ipynb](https://github.com/TheJacksonLaboratory/sbas/blob/master/jupyter/figure1.ipynb)). We have archived the results from the notebook that generates the results using the method described by the author of the R package [`ropensci/piggyback`](https://github.com/ropensci/piggyback). We use the release named `dge` (Differential Gene Expression) in the repo and can be accessed at [TheJacksonLaboratory/sbas/releases/tag/dge](https://github.com/TheJacksonLaboratory/sbas/releases/tag/dge). 

For using the [`ropensci/piggyback`](https://github.com/ropensci/piggyback) with private repositories, it is required that a `GITHUB_TOKEN` is stored as a variable in the R environment in which one is working. To generate such a token with sensible default permissions, the R package [usethis]() has a convenient function 

```R
# intall.packages("usethis")
usethis::browse_github_token()
```

This will redirect you to GitHub to create your own GitHub token. Once you have the token, you can use it to set up `.Renviron` by typing the following:

```R
Sys.setenv(GITHUB_TOKEN = "youractualtokenindoublequotes")
```

Then you sre ready to use the function [`piggyback::pb_download()`](https://docs.ropensci.org/piggyback/reference/pb_download.html) to retrieve the `dge.tar.gz` that contains the topTable objects written as .csv file for all 46 examined GTEX tissue cohorts.

---

***NOTE***

Avoid using the `.token` argument to share your token directly in the function as you might forget and push your code, along with your private GITHUB_TOKEN to GitHub. If that happens by mistake, it is advised you invalidate the token that has been exposed by accessing [this link](https://github.com/settings/tokens) and clicking `Delete`.

---

In [2]:
#?piggyback::pb_download()

In [3]:
if (!file.exists("../data/DGE_gene_csv.tar.gz")) {
    
    message("Fetching dge.tar.gz from GitHub ..")
    # Download archive from GitHub release with tag "dge"
    piggyback::pb_download(file = "DGE_gene_csv.tar.gz",
                           dest = "../data",
                           repo = "TheJacksonLaboratory/sbas",
                           tag  = "dge",
                           show_progress = TRUE)
    message("Done!\n")
    
    message("Decompressing archive into folder ../data/dge ..")
    # Decompress in a folder tmp named dge
    system("mkdir -p ../data/dge && tar xvzf ../data/DGE_gene_csv.tar.gz -C ../data/dge/", intern = TRUE)
    message("Done!\n")
} else {
    message("File dge.tar.gz already available in ../data/ !\n")
    message("Decompressing archive into folder ../data/dge ..")
    # Decompress in a folder tmp named dge
    system("mkdir -p ../data/dge && tar xvzf ../data/DGE_gene_csv.tar.gz -C ../data/dge/", intern = TRUE)
    message("Done!\n")
}


File dge.tar.gz already available in ../data/ !


Decompressing archive into folder ../data/dge ..

Done!




## To get the last column fields of the GTF file to annotate the gene ids
- The following code cells download the Ensembl or Gencode GTF file (for now we are using Ensembl). Alternatively, you can get the files from the command line as follows.

```bash
wget ftp://ftp.ensembl.org/pub/release-100/gtf/homo_sapiens/Homo_sapiens.GRCh38.100.chr_patch_hapl_scaff.gtf.gz -P ../data
wget ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_30/gencode.v30.annotation.gtf.gz -P ../data
```

In [4]:
# Download from archive if not available in ../data
if (!("Homo_sapiens.GRCh38.100.chr_patch_hapl_scaff.gtf.gz" %in% list.files("../data/"))) {
    message("Downloading Homo_sapiens.GRCh38.100.chr_patch_hapl_scaff.gtf.gz from Ensembl into the ../data/ directory ..\n")
    gtf_url = 'ftp://ftp.ensembl.org/pub/release-100/gtf/homo_sapiens/Homo_sapiens.GRCh38.100.chr_patch_hapl_scaff.gtf.gz'
    destination = '../data/Homo_sapiens.GRCh38.100.chr_patch_hapl_scaff.gtf.gz'
    download.file(gtf_url, destination, "wget", quiet = FALSE, mode = "w",
              cacheOK = TRUE,
              extra = getOption("download.file.extra"))
    message("Done!\n")
} else {
    message("GTF file previously downloaded")
}

GTF file previously downloaded



In [5]:
# ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_30/gencode.v30.chr_patch_hapl_scaff.annotation.gtf.gz
if (!("gencode.v30.chr_patch_hapl_scaff.annotation.gtf.gz" %in% list.files("../data/"))) {
    message("Downloading gencode.v30.chr_patch_hapl_scaff.annotation.gtf.gz from Gencode into the ../data/ directory ..\n")
    gencode_url = 'ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_30/gencode.v30.chr_patch_hapl_scaff.annotation.gtf.gz'
    destination = '../data/gencode.v30.chr_patch_hapl_scaff.annotation.gtf.gz'
    download.file(gencode_url, destination, "wget", quiet = FALSE, mode = "w",
            cacheOK = TRUE,
            extra = getOption("download.file.extra"))
    message("Done!\n")
} else {
    message("Genecode file previously downloaded")
}

Genecode file previously downloaded



## Mapping the Ensembl geneids to gene symbols

The `limma::topTable()` dataframes encode the feature information (genes) as rownames of the dataframe. The values are Ensembl Gene IDs. To map the Gene ids to Gene symbold we will use a the fields in the last column of the relevant GTF file. 

Note: If you are using GENCODE to retrieve the GTF file, use the relevant field name for the gene ID:

| Source | Gene Identifier Name |
|---:|:---|
| GENCODE|`Geneid`|
| Ensembl|`gene_id`|




- To parse the GTF file and retrieve the fields associated with the Gene IDs, Gene Symbols etc you could use the following bash snippet, depending on the source of your GTF (Ensembl, GENCODE) as these two files slightly differ.

```bash
# found here: https://www.biostars.org/p/140471/
cd sbas/data

zcat gencode.v30.annotation.gtf.gz | awk 'BEGIN{FS="\t"}{split($9,a,";"); if($3~"gene") print a[1]"\t"a[3]"\t"$1":"$4"-"$5"\t"a[2]"\t"$7}' |sed 's/gene_id "//' | sed 's/gene_id "//' | sed 's/gene_type "//'| sed 's/gene_name "//' | sed 's/"//g' | awk 'BEGIN{FS="\t"}{split($3,a,"[:-]"); print $1"\t"$2"\t"a[1]"\t"a[2]"\t"a[3]"\t"$4"\t"$5"\t"a[3]-a[2];}' | sed "1i\Geneid\tGeneSymbol\tChromosome\tStart\tEnd\tClass\tStrand\tLength" > gencode.v30.annotation.gtf.gz.txt

zcat Homo_sapiens.GRCh38.100.chr_patch_hapl_scaff.gtf.gz | awk 'BEGIN{FS="\t"}{split($9,a,";"); if($3~"gene") print a[1]"\t"a[3]"\t"$1":"$4"-"$5"\t"a[5]"\t"$7}' | sed 's/gene_id "//' | sed 's/gene_id "//' | sed 's/gene_biotype "//'| sed 's/gene_name "//' | sed 's/gene_biotype "//' | sed 's/"//g' | sed 's/ //g' | sed '1igene_id\tGeneSymbol\tChromosome\tClass\tStrand' > Homo_sapiens.GRCh38.100.chr_patch_hapl_scaff.gtf.gz.txt 
```

In [6]:
# The following awesome AWK command retrieves the fields we want from the Ensembl GTF file
zcat_command = 'zcat Homo_sapiens.GRCh38.100.chr_patch_hapl_scaff.gtf.gz | awk \'BEGIN{FS="\t"}{split($9,a,";"); if($3~"gene") print a[1]"\t"a[3]"\t"$1":"$4"-"$5"\t"a[5]"\t"$7}\' | sed \'s/gene_id "//\' | sed \'s/gene_id "//\' | sed \'s/gene_biotype "//\'| sed \'s/gene_name "//\' | sed \'s/gene_biotype "//\' | sed \'s/"//g\' | sed \'s/ //g\' | sed \'1igene_id\tGeneSymbol\tChromosome\tClass\tStrand\' > Homo_sapiens.GRCh38.100.chr_patch_hapl_scaff.gtf.gz.txt '
command = paste('cd ../data/; ', zcat_command, sep=' ')
system(command)

In [7]:
# Now do the same for gencode
zcat_command = 'zcat gencode.v30.annotation.gtf.gz | awk \'BEGIN{FS="\t"}{split($9,a,";"); if($3~"gene") print a[1]"\t"a[3]"\t"$1":"$4"-"$5"\t"a[2]"\t"$7}\' |sed \'s/gene_id "//\' | sed \'s/gene_id "//\' | sed \'s/gene_type "//\'| sed \'s/gene_name "//\' | sed \'s/"//g\' | awk \'BEGIN{FS="\t"}{split($3,a,"[:-]"); print $1"\t"$2"\t"a[1]"\t"a[2]"\t"a[3]"\t"$4"\t"$5"\t"a[3]-a[2];}\' | sed "1i\tGeneid\tGeneSymbol\tChromosome\tStart\tEnd\tClass\tStrand\tLength" > gencode.v30.annotation.gtf.gz.txt'
command = paste('cd ../data/; ', zcat_command, sep=' ')
system(command)

## Preview the GTF tables with the gene attributes

To make sure the snippets above have worked as expected, take a look in the tables with `head()`:

In [8]:
ensembl_path <- "Homo_sapiens.GRCh38.100.chr_patch_hapl_scaff.gtf.gz.txt"
gencode_path <- "gencode.v30.annotation.gtf.gz.txt"
gtf_ensembl <- read.table(paste0("../data/", ensembl_path), header = TRUE)
gtf_gencode <- read.table(paste0("../data/", gencode_path), header = TRUE)

head(gtf_ensembl,2)
head(gtf_gencode,2)

,gene_id,GeneSymbol,Chromosome,Class,Strand
,<fct>,<fct>,<fct>,<fct>,<fct>
1,ENSG00000223972,DDX11L1,1:11869-14409,transcribed_unprocessed_pseudogene,+
2,ENSG00000227232,WASH7P,1:14404-29570,unprocessed_pseudogene,-


,Geneid,GeneSymbol,Chromosome,Start,End,Class,Strand,Length
,<fct>,<fct>,<fct>,<int>,<int>,<fct>,<fct>,<int>
1,ENSG00000223972.5,DDX11L1,chr1,11869,14409,transcribed_unprocessed_pseudogene,+,2540
2,ENSG00000227232.5,WASH7P,chr1,14404,29570,unprocessed_pseudogene,-,15166


# Create a list of named dataframes with the Differential Gene Expression `limma::topTable()`s

We will iterate over the list of named dataframes to collect summary statistics. More specifically, retrieve the count of:
- upregulated
- downregulated
- non significant

genes for the contrast males-females per tissue.

In [9]:
dge_tables_filepaths <- list.files("../data/dge/", pattern = "*DGE.csv", full.names = TRUE)
dge_tables_filenames <- list.files("../data/dge/", pattern = "*DGE.csv", full.names = FALSE)

In [10]:
all_topTables <- lapply(dge_tables_filepaths,read.csv)
names(all_topTables) <- gsub("_DGE.csv","", dge_tables_filenames, fixed = TRUE)

The list named `all_topTables` is the object that holds all the topTable dataframes from each tissue comparison:
There should be `39` tissues

In [11]:
length(all_topTables)

[1] 39

In [12]:
summary(all_topTables)

                                      Length Class      Mode
adipose_subcutaneous                  6      data.frame list
adipose_visceral_omentum              6      data.frame list
adrenal_gland                         6      data.frame list
artery_aorta                          6      data.frame list
artery_coronary                       6      data.frame list
artery_tibial                         6      data.frame list
brain_caudate_basal_ganglia           6      data.frame list
brain_cerebellar_hemisphere           6      data.frame list
brain_cerebellum                      6      data.frame list
brain_cortex                          6      data.frame list
brain_frontal_cortex_ba_9             6      data.frame list
brain_hippocampus                     6      data.frame list
brain_hypothalamus                    6      data.frame list
brain_nucleus_accumbens_basal_ganglia 6      data.frame list
brain_putamen_basal_ganglia           6      data.frame list
brain_spinal_cord_cervic

# Trim gene versions from gene names to match Gene Identifiers from GTF (GENCODE, Ensembl)
Remove characters after `.` in the Gene Identifier column, since as long as the gene version information is present we will not be able to perform a join, to annotate the toptable dataframes with the GTF gene attrubutes.

In [13]:
GTF_SOURCE <- "ensembl"      # c("gencode", "ensembl")
GENE_ID    <- "gene_id"      # c("Geneid" , "gene_id")

#GTF_SOURCE <- "gencode"      # c("gencode", "ensembl")
#GENE_ID    <- "Geneid"      # c("Geneid" , "gene_id")

if (GTF_SOURCE == "gencode") { GTF <- gtf_gencode}
if (GTF_SOURCE == "ensembl") { GTF <- gtf_ensembl}

GTF[[GENE_ID]]       <- gsub("\\..*","", GTF[[GENE_ID]])

# Example with one topTable before iterating over all tissues

In [14]:
# Example topTable and name
topTable <- all_topTables[[1]]
name     <- names( all_topTables)[1]
topTable[[GENE_ID]]  <- gsub("\\..*","", rownames(topTable))
name
head(topTable,2)
head(GTF, 2)

[1] "adipose_subcutaneous"

,logFC,AveExpr,t,P.Value,adj.P.Val,B,gene_id
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
ENSG00000176728.7,-7.748207,-1.0323929,-104.4253,4.803353e-308,2.680943e-303,671.1671,ENSG00000176728
ENSG00000260197.1,-8.474073,-0.7043022,-101.6645,3.159370e-303,8.816855e-299,663.9100,ENSG00000260197


,gene_id,GeneSymbol,Chromosome,Class,Strand
,<chr>,<fct>,<fct>,<fct>,<fct>
1,ENSG00000223972,DDX11L1,1:11869-14409,transcribed_unprocessed_pseudogene,+
2,ENSG00000227232,WASH7P,1:14404-29570,unprocessed_pseudogene,-


## Defining the thresholds for the double criterion filtering:

Criteria:
- Adjusted p-value < `p_value_cuttoff`
- Absolute FoldChange > `absFold_change_threshold`

----

***NOTE***

Defining higher in males or females based on the limma design matrix.
As we have used 1 for encoding the females and 2 for the males, our *reference level* for the contrast in the expression between males and females is 1, the females.


From the `limma` documentation:
>The level which is chosen for the *reference level* is the level which is contrasted against. By default, this is simply the first level alphabetically. We can specify that we want group 2 to be the reference level by either using the relevel function [..]

By convention, we could say that genes with positive log fold change, are higher in males, whereas the opposite holds true for the ones that are observed to have negative log fold change. 

---

In [15]:
adj.P.Val_threshold  <- 0.5
absFoldChange_cutoff <- 1.5

Replacing potential `NA` values in the `P.Value`, `adj.P.Val` to keep the columns numeric and avoid coersion.

In [16]:
# replacing NA p-values with p-value = 1
topTable$P.Value[is.na(topTable$P.Value)]     <- 1; 
topTable$adj.P.Val[is.na(topTable$adj.P.Val)] <- 1;

In [17]:
# Add helper variable dummy `FoldChange` variable. Use 2 as base of log, because this is the default from limma
# The following statement calculates a dummy fold change (how many times higher or lower)
# The minus symbol is a convention symbol only! to express eg. a fold change of 0.25 as -4, 4 times lower
topTable$FoldChange_dummy    <-   ifelse(topTable$logFC > 0, 2 ^ topTable$logFC, -1 / (2 ^ topTable$logFC))                    

# Add helper variable `abs_logFC`.
topTable$abs_logFC <- abs(topTable$logFC)

# Add helper variable `abundance` for up, down, non_signif
topTable$abundance                                                  <- "non_signif"
topTable$abundance[(topTable$logFC >=   log2(absFoldChange_cutoff)) & (topTable$adj.P.Val <= adj.P.Val_threshold )]   <- "higher"
topTable$abundance[(topTable$logFC <=  -log2(absFoldChange_cutoff)) & (topTable$adj.P.Val <= adj.P.Val_threshold )]   <- "lower"

In [18]:
head(topTable)

,logFC,AveExpr,t,P.Value,adj.P.Val,B,gene_id,FoldChange_dummy,abs_logFC,abundance
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>
ENSG00000176728.7,-7.748207,-1.0323929,-104.42525,4.803353e-308,2.680943e-303,671.1671,ENSG00000176728,-215.00207,7.748207,lower
ENSG00000260197.1,-8.474073,-0.7043022,-101.66447,3.159370e-303,8.816855e-299,663.9100,ENSG00000260197,-355.59053,8.474073,lower
ENSG00000231535.5,-5.967284,-2.8393448,-100.45548,4.441637e-301,8.263518e-297,662.6696,ENSG00000231535,-62.56498,5.967284,lower
ENSG00000229236.1,-6.004324,-2.8556966,-97.58577,6.958504e-296,7.767639e-292,651.9625,ENSG00000229236,-64.19212,6.004324,lower
ENSG00000129824.15,-9.477673,4.5387832,-94.53313,3.342811e-290,3.109594e-286,632.7641,ENSG00000129824,-712.95772,9.477673,lower
ENSG00000067646.11,-9.267686,0.6631333,-98.59282,1.008611e-297,1.407365e-293,631.6809,ENSG00000067646,-616.38424,9.267686,lower


# Define a vector with the columns to keep in the annotated from GTF `topTable` object

In [19]:
toKeep <- c("Geneid","logFC","FoldChange_dummy", "adj.P.Val", "abundance")

In [20]:
head(topTable[ , colnames(topTable) %in% toKeep ],2)

,logFC,adj.P.Val,FoldChange_dummy,abundance
,<dbl>,<dbl>,<dbl>,<chr>
ENSG00000176728.7,-7.748207,2.680943e-303,-215.0021,lower
ENSG00000260197.1,-8.474073,8.816855e-299,-355.5905,lower


In [21]:
name
dim(topTable)
dim(topTable [topTable$abundance != "non_signif",  ])
dim(topTable [ (topTable$abundance != "non_signif" )  & (topTable$adj.P.Val <= adj.P.Val_threshold ) ,  ])

[1] "adipose_subcutaneous"

[1] 55814    10

[1] 638  10

[1] 638  10

In [22]:
expression_abundance <- t(table(topTable$abundance))
expression_abundance

      
       higher lower non_signif
  [1,]    312   326      55176

In [23]:
expression_abundance <- t(table(topTable$abundance))
signif <- as.data.frame.matrix(expression_abundance)

In [24]:
signif

higher,lower,non_signif
<int>,<int>,<int>
312,326,55176


To avoid errors in the cases that we might have none lower or none higher, and the matrix might be missing columns we will create a template data.frame and also add the column that might be missing if lower or higher genes is equal to 0.

In [25]:
signif_template <- structure(list(higher = integer(0), 
                                   lower = integer(0), 
                                   non_signif = integer(0)), 
                              row.names = integer(0), class = "data.frame")
signif_template

higher,lower,non_signif
<int>,<int>,<int>


In the for-loop we will check if both columns `lower`, `higher` are present, if not add the column and zero count to create the expected shape of the dataframe:

```R
signif <- as.data.frame.matrix(expression_abundance)
if(! ("higher" %in% colnames(signif))) { 
    
    signif$higher <- 0
}
if(! ("lower" %in% colnames(signif))) { 

    signif$lower <- 0
}
```

Now we can add some more summary statistics eg percentage of genes lower, higher or non-significantly different, 

In [26]:
signif$tissue <- name
signif$sum    <- signif$non_signif + signif$higher + signif$lower
toKeepInOrder <- c("tissue", "non_signif", "lower", "higher", "% lower", "% higher", "% non-signif")
signif$`% higher`     <-  round(signif$higher / signif$sum  * 100, 2)
signif$`% lower`      <-  round(signif$lower / signif$sum  * 100, 2)
signif$`% non-signif` <-  round(signif$non_signif / signif$sum  * 100, 2)
signif <- signif[, toKeepInOrder]
signif

tissue,non_signif,lower,higher,% lower,% higher,% non-signif
<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
adipose_subcutaneous,55176,326,312,0.58,0.56,98.86


# Summary table of differentially expressed genes between male and female acrosss tissues

Above we demonstrate for one example limma `topTable`. Let's now iterate over all tissue and create an aggregated table of counts of differentially expressed or non-significantly altered between the two sexes.

In [27]:
summary_signif <-structure(list(tissue = character(0), 
                            non_signif = integer(0), 
                            lower = integer(0),
                            higher = integer(0),
                            `% lower` = numeric(0), 
                            `% higher` = numeric(0), 
                            `% non-signif` = numeric(0)), 
                       row.names = integer(0), 
                       class = "data.frame")

signif_template <- structure(list(higher = integer(0), 
                                   lower = integer(0), 
                                   non_signif = integer(0)), 
                              row.names = integer(0), class = "data.frame")

signif_per_tissue <- structure(list(logFC = numeric(0), AveExpr = numeric(0), t = numeric(0), 
                        P.Value = numeric(0), adj.P.Val = numeric(0), B = numeric(0), 
                        initial_gene_id = character(0), gene_id = character(0), abs_logFC = numeric(0), 
                        FoldChange_dummy = numeric(0), abundance = character(0), 
                        GeneSymbol = character(0), Chromosome = character(0), Class = character(0), 
                        Strand = character(0), tissue = character(0)), row.names = integer(0), class = "data.frame")


for (i in seq_along(all_topTables)){
    topTable <- all_topTables[[i]]
    name     <- names(all_topTables)[i] 
    initial_gene_id <- paste0("initial_", GENE_ID)
    topTable[[initial_gene_id]] <- rownames(topTable)
    topTable[[GENE_ID]]  <- gsub("\\..*","", rownames(topTable))
    # replacing NA p-values with p-value = 1
    topTable$P.Value[is.na(topTable$P.Value)]     <- 1; 
    topTable$adj.P.Val[is.na(topTable$adj.P.Val)] <- 1;
    topTable$abs_logFC <- abs(topTable$logFC)
    # Add helper variable dummy `FoldChange` variable. Use 2 as base of log, because this is the default from limma
    # The following statement calculates a dummy fold change (how many times higher or lower)
    # The minus symbol is a convention symbol only! to express eg. a fold change of 0.25 as -4, 4 times lower
    topTable$FoldChange_dummy    <-   ifelse(topTable$logFC > 0, 2 ^ topTable$logFC, -1 / (2 ^ topTable$logFC))                    

    # Add helper variable `abs_logFC`.
    topTable$abs_logFC <- abs(topTable$logFC)

    # Add helper variable `abundance` for up, down, non_signif
    topTable$abundance                                                  <- "non_signif"
    topTable$abundance[(topTable$logFC >   log2(absFoldChange_cutoff)) & (topTable$adj.P.Val <= adj.P.Val_threshold )]   <- "higher"
    topTable$abundance[(topTable$logFC <  -log2(absFoldChange_cutoff)) & (topTable$adj.P.Val <= adj.P.Val_threshold )]   <- "lower"
    before_gtf_merge <- dim(topTable)[1]
    topTable <- dplyr::left_join(topTable, GTF, by = GENE_ID)
    topTable_signif <- topTable[ topTable$abundance != "non_signif", ]
    topTable_signif$tissue <- name
    signif_per_tissue <- rbind(signif_per_tissue, topTable_signif )
    data.table::fwrite(file = paste0("../data/signif_", snakecase::to_snake_case(name), ".csv"), topTable_signif)
    after_gtf_merge <- dim(topTable)[1]
    message( name, ", N features before GTF merge: ",before_gtf_merge, ", after: ", after_gtf_merge)
    expression_abundance <- t(table(topTable$abundance))
    signif <- as.data.frame.matrix(expression_abundance)
    if(! ("higher" %in% colnames(signif))) {
        signif$higher <- 0
    }
    if(! ("lower" %in% colnames(signif))) {
        signif$lower <- 0
    }
    signif$tissue <- name
    signif$sum    <-   signif$non_signif + signif$higher + signif$lower
    toKeepInOrder <- c("tissue", "non_signif", "lower", "higher", "% lower", "% higher", "% non-signif")
    signif$`% higher`     <-  round(signif$higher / signif$sum  * 100, 2)
    signif$`% lower`      <-  round(signif$lower / signif$sum  * 100, 2)
    signif$`% non-signif` <-  round(signif$non_signif / signif$sum  * 100, 2)
    signif <- signif[, toKeepInOrder]
    summary_signif <- rbind(summary_signif, signif)   
}

summary_signif <- summary_signif[order(summary_signif$`% non-signif`), ]
head(summary_signif , 2)
head(signif_per_tissue, 2)

adipose_subcutaneous, N features before GTF merge: 55814, after: 55814

adipose_visceral_omentum, N features before GTF merge: 55814, after: 55814

adrenal_gland, N features before GTF merge: 55814, after: 55814

artery_aorta, N features before GTF merge: 55814, after: 55814

artery_coronary, N features before GTF merge: 55814, after: 55814

artery_tibial, N features before GTF merge: 55814, after: 55814

brain_caudate_basal_ganglia, N features before GTF merge: 55814, after: 55814

brain_cerebellar_hemisphere, N features before GTF merge: 55814, after: 55814

brain_cerebellum, N features before GTF merge: 55814, after: 55814

brain_cortex, N features before GTF merge: 55814, after: 55814

brain_frontal_cortex_ba_9, N features before GTF merge: 55814, after: 55814

brain_hippocampus, N features before GTF merge: 55814, after: 55814

brain_hypothalamus, N features before GTF merge: 55814, after: 55814

brain_nucleus_accumbens_basal_ganglia, N features before GTF merge: 55814, after: 558

,tissue,non_signif,lower,higher,% lower,% higher,% non-signif
,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
17,breast_mammary_tissue,49656,1981,4177,3.55,7.48,88.97
16,brain_spinal_cord_cervical_c_1,52471,175,3168,0.31,5.68,94.01


,logFC,AveExpr,t,P.Value,adj.P.Val,B,initial_gene_id,gene_id,abs_logFC,FoldChange_dummy,abundance,GeneSymbol,Chromosome,Class,Strand,tissue
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<fct>,<fct>,<fct>,<fct>,<chr>
1,-7.748207,-1.0323929,-104.4253,4.803353e-308,2.680943e-303,671.1671,ENSG00000176728.7,ENSG00000176728,7.748207,-215.0021,lower,TTTY14,Y:18772706-19077563,lncRNA,-,adipose_subcutaneous
2,-8.474073,-0.7043022,-101.6645,3.159370e-303,8.816855e-299,663.9100,ENSG00000260197.1,ENSG00000260197,8.474073,-355.5905,lower,AC010889.1,Y:19691941-19694606,lncRNA,-,adipose_subcutaneous


# Defining higher in males or females based on the limma design matrix
As we have used 1 for encoding the males and 2 for the females, our *reference level* for the contrast in the expression between males and females is 1, the males.


From the `limma` documentation:
>The level which is chosen for the *reference level* is the level which is contrasted against. By default, this is simply the first level alphabetically. We can specify that we want group 2 to be the reference level by either using the relevel function [..]

By convention, we could say that genes with positive log fold change, are higher in females, whereas the opposite holds true for the ones that are observed to have negative log folde change. 

In [28]:
summary_signif$`higher in males`   <- summary_signif$lower
summary_signif$`higher in females` <- summary_signif$higher
head(summary_signif[summary_signif$tissue == "Adipose-Subcutaneous", ])

tissue,non_signif,lower,higher,% lower,% higher,% non-signif,higher in males,higher in females
<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>


# Preparing the summary table for plotting

We will need to aggregate the number of genes in one column in order to be able to plot, and also convert the `Tissue` column to a factor. We will use the `reshape` R package to *melt* the dataframe from a wide to a long version, as described above:

In [ ]:
toPlot <- summary_signif[, c( "tissue", "higher in males", "higher in females")]
toPlot <- reshape::melt(toPlot, id=c("tissue"))
toPlot$tissue <- as.factor(toPlot$tissue)
colnames(toPlot) <- c("Tissue", "Sex Bias", "Number of Genes")
head(toPlot[toPlot$Tissue == "Adipose-Subcutaneous", ])

In [ ]:

#options(repr.plot.width=15.5, repr.plot.height=20)

ggplot(toPlot, aes(x = Tissue, y = `Number of Genes`, fill = `Sex Bias`)) + 
  geom_bar(stat="identity", position = "dodge") + 
  scale_fill_manual (values = c( "higher in males" = "#4A637B" , "higher in females" = "#f35f71")) + 
  
  theme(text              = element_text(color = "#4A637B", face = "bold", family = 'Helvetica')
        ,plot.caption     = element_text(size =  12, color = "#8d99ae", face = "plain", hjust= 1.05) 
        ,plot.title       = element_text(size =  18, color = "#2b2d42", face = "bold", hjust= 0.5)
        ,axis.text.y      = element_text(angle =  0, size = 10, color = "#8d99ae", face = "bold", hjust=1.1)
        ,axis.text.x      = element_text(angle = 70, size = 12, color = "#8d99ae", face = "bold", hjust=1.1)
        ,axis.title.x     = element_blank()
        ,axis.ticks.x     = element_blank()
        ,axis.ticks.y     = element_blank()
        ,plot.margin      = unit(c(1,1,1,1),"cm")
        ,panel.background = element_blank()
        ,legend.position  = "right") +
  

  geom_text(aes(y = `Number of Genes` + 15, 
                label = `Number of Genes`),
                size = 3,
                color     = "#4A637B",
                position  =  position_dodge(width = 1),
                family    = 'Helvetica') +
  
  labs(title   = "Number of genes with higher expression in each sex per tissue\n",
       caption = "\nsource: 'The impact of sex on alternative splicing'\n doi: https://doi.org/10.1101/490904",
       y   = "\nNumber of Differentially Expressed Genes")  + coord_flip()



# Mutually exclusive sex biased genes (higher expression in one or the other sex only)


The dataframe `signif_per_tissue` contains all the information for the genes that were significantly higher in either of the two sexes. WLet's examine how many mutually exclusive genes were found across all examined tissues. Ensembl encodes as `Chromosome` the chromosomal position, so we will create the required variables to retrieve only the chromosome information for producing summary statistics.

In [ ]:
dput(colnames(signif_per_tissue))

In [ ]:
signif_per_tissue$Chromosomal_Position <- signif_per_tissue$Chromosome
signif_per_tissue$Chromosome <- gsub("\\:.*","", signif_per_tissue$Chromosome)
signif_per_tissue$higher_in  <- 0
signif_per_tissue$higher_in[(signif_per_tissue$abundance == "lower" )] <- "males"
signif_per_tissue$higher_in[(signif_per_tissue$abundance == "higher" )] <- "females"
toKeepInOrder <- c( paste0("initial_", GENE_ID), "GeneSymbol", "logFC",  "adj.P.Val", "abundance", "higher_in",  "tissue", "Chromosome", 
GENE_ID, "abs_logFC", "FoldChange_dummy", 
"Class", "Strand","Chromosomal_Position", 
 "AveExpr", "t", "P.Value", "adj.P.Val", "B")
signif_per_tissue <- signif_per_tissue[, toKeepInOrder]
head(signif_per_tissue, 4)

# Examine mutually exclusive genes upregulated in each sex

In [ ]:
female_biased <- unique(signif_per_tissue[[paste0("initial_", GENE_ID)]] [ signif_per_tissue$higher_in == "females" ] )
male_biased   <- unique(signif_per_tissue[[paste0("initial_", GENE_ID)]] [ signif_per_tissue$higher_in == "males"  ] )

length(male_biased)
length(female_biased)

In [ ]:
## Present in both

length((intersect(male_biased, female_biased)))
length((intersect(female_biased, male_biased)))

intersect <- (intersect(male_biased, female_biased))


In [ ]:
## Only in males
length(male_biased[! (male_biased %in% intersect)])

## Only females
length(female_biased[! (female_biased %in% intersect)])

In [ ]:
perc_only_male <-  length(male_biased[! (male_biased %in% intersect)]) / length(male_biased) * 100
perc_only_female <-  length(female_biased[! (female_biased %in% intersect)]) / length(female_biased) * 100

head( signif_per_tissue[ signif_per_tissue[[paste0("initial_", GENE_ID)]] %in% male_biased[! (male_biased %in% intersect)],  ] , 4 )


message(round(perc_only_male, 2), " % of differentially expressed genes higher in males only found to be significantly differentin males")
message(round(perc_only_female,2), " % of differentially expressed genes higher in females only found to be significantly different in females")

## Significantly higher only in males

In [ ]:
dim(signif_per_tissue[ signif_per_tissue[[paste0("initial_", GENE_ID)]] %in% male_biased[! (male_biased %in% intersect)],  ])

only_male_genes <- signif_per_tissue[ signif_per_tissue[[paste0("initial_", GENE_ID)]] %in% (male_biased[! (male_biased %in% intersect)]) ,  ]

head(only_male_genes[ order(only_male_genes[[paste0("initial_", GENE_ID)]] ), ], 5)

In [ ]:
# See 8.1.1 enquo() and !! - Quote and unquote arguments in https://tidyeval.tidyverse.org/dplyr.html

only_male_genes %>% 
    count( !!GENE_ID, GeneSymbol, Class, sort = TRUE) %>%
    head(20)

## Significantly higher only in females

In [ ]:
only_female_genes <- signif_per_tissue[ signif_per_tissue[[paste0("initial_", GENE_ID)]] %in% (female_biased[! (female_biased %in% intersect)]) ,  ]

head(only_female_genes[ order(only_female_genes[[paste0("initial_", GENE_ID)]] ), ], 10)

In [ ]:
only_female_genes %>% 
    count( !!GENE_ID, GeneSymbol, Class, sort = TRUE) %>%
    head(20)

# Examine number of differentially expressed genes per chromosome per sex

In [ ]:
signif_per_tissue$Chromosome <- as.factor(signif_per_tissue$Chromosome)
signif_per_tissue$higher_in <- as.factor(signif_per_tissue$higher_in)

signif_per_tissue %>% 
    group_by(Chromosome,higher_in) %>%  
    count()  -> signif_per_chrom_per_sex

In [ ]:
signif_per_chrom_per_sex

## Metadata

For replicability and reproducibility purposes, we also print the following metadata:

1. Checksums of **'artefacts'**, files generated during the analysis and stored in the folder directory **`data`**
2. List of environment metadata, dependencies, versions of libraries using `utils::sessionInfo()` and [`devtools::session_info()`](https://devtools.r-lib.org/reference/session_info.html)

In [ ]:
notebook_id   = "summary_per_tissue_diff_expressed"

message("Generating sha256 checksums of the artefacts in the `..data/` directory .. ")
system(paste0("cd ../data/ && sha256sum **/*csv > ../metadata/", notebook_id, "_sha256sums.txt"), intern = TRUE)
system(paste0("cd ../data/ && sha256sum *csv >> ../metadata/", notebook_id, "_sha256sums.txt"), intern = TRUE)

message("Done!\n")

data.table::fread(paste0("../metadata/", notebook_id, "_sha256sums.txt"), header = FALSE, col.names = c("sha256sum", "file"))

### 2. Libraries metadata

In [ ]:
dev_session_info   <- devtools::session_info()
utils_session_info <- utils::sessionInfo()

message("Saving `devtools::session_info()` objects in ../metadata/devtools_session_info.rds  ..")
saveRDS(dev_session_info, file = paste0("../metadata/", notebook_id, "_devtools_session_info.rds"))
message("Done!\n")

message("Saving `utils::sessionInfo()` objects in ../metadata/utils_session_info.rds  ..")
saveRDS(utils_session_info, file = paste0("../metadata/", notebook_id ,"_utils_info.rds"))
message("Done!\n")

dev_session_info$platform
dev_session_info$packages[dev_session_info$packages$attached==TRUE, ]

# Calculating the sex-biased splicing index
The normalized sex-biased splicing index is defined as the number of statistically significant splicing events per 1000 exons in the chromosome.

In [ ]:
dim(signif_per_chrom_per_sex)

Sorry, I cannot do this in R. Here is Python (ugly script but works)

import csv
import gzip
import re
from collections import defaultdict

fname = 'Homo_sapiens.GRCh38.100.chr_patch_hapl_scaff.gtf.gz'

chrom2exons = defaultdict(set)

with gzip.open(fname, 'rt') as f:
    cr = csv.reader(f, delimiter='\t', quotechar='"')
    for row in cr:
        #print(row)
        if row[0].startswith('#'):
            continue
        chrom = row[0]
        annots = row[8]
        fields = annots.split(";")
        exon = re.compile(r'exon_id "(ENSE\d+)"')
        for f in fields:
            itm = f.strip()
            match = exon.match(itm)
            if match:
                exonid = match.group(1)
                chrom2exons[chrom].add(exonid)

g = open('chrom2exons.txt', 'wt')
for k, v in chrom2exons.items():
    print("chr{}: n={}".format(k, len(v)))
    g.write("{}\t{}\n".format(k, len(v)))
g.close()

1	69381
2	55599
3	46452
4	29749
5	34789
6	33817
7	35973
X	22471
8	28489
9	26460
11	43212
10	26514
12	42925
13	13193
14	25994
15	28720
16	36285
17	45142
18	13360
20	16704
19	44166
Y	2908
22	16411
21	8830
MT	37

In [ ]:
signif_per_chrom_per_sex

In [ ]:
only_female_genes %>% 
    count( !!GENE_ID, GeneSymbol, Class, sort = TRUE) %>%
    head(20)

In [ ]:
signif_per_tissue %>% 
     group_by(Chromosome) %>%  
    count()  -> signif_per_chrom

In [ ]:
signif_per_chrom

In [ ]:
chrom2exon_filename = '../assets/canon_chrom2exons.txt'
if (! file.exists(chrom2exon_filename)) {
    message("Could not find canon_chrom2exons.txt file")
}
c2e_df = read.csv(chrom2exon_filename, sep='\t', header=FALSE)
colnames(c2e_df) <- c("Chromosome","exons")
head(c2e_df) # 25 chromosomes including MT

In [ ]:
df2 <- merge(signif_per_chrom, c2e_df, by="Chromosome")
head(df2)

In [ ]:
# calculate splicinig index
library(tidyverse)

In [ ]:
df2 %>% 
  mutate(Index = 1000 * n/exons) -> df3

In [ ]:
df4 <- df3[-25,] # remove the Y chromosome
df4 <- df4[-23,] # remove the MT chromosome

In [ ]:
rm(res_sorted)
res_sorted <- df4[order(df4$Index, decreasing=TRUE),]
res_sorted


In [ ]:
res_sorted$Chromosome <- factor(res_sorted$Chromosome, levels = res_sorted$Chromosome)
res_sorted

In [ ]:
# set the colors
npgBlue<- rgb(60/256,84/256,136/256,1)
npgRed <- rgb(220/256,0,0,0.5)
npgGreen <- rgb(0,160/256,135/256,1)
npgBrown <- rgb(126/256,97/256,72/256,1)

In [ ]:
# make the plot 
figure2b <- ggplot(res_sorted, aes(x = Chromosome, y = Index, size = n)) +
  geom_point(color=npgBlue) +
  theme_bw() +
  theme(axis.text.x = element_text(size=14, angle = 270, hjust = 0.0, vjust = 0.5),
	axis.text.y = element_text(size=16),
	axis.title.x = element_blank(),
	axis.title.y = element_text(face="plain", colour="black",
                                    size=18),
	legend.title=element_blank(),
	legend.text = element_text(face="plain", colour="black",
                                   size=14)) +
  scale_fill_viridis_c() +
  ylab(paste("Sex-biased splicing index ")) +
  xlab("Chromosomes") +
  guides(size = guide_legend(title = "Number of ASE"))
figure2b
